In [1]:
import folium
from IPython.display import HTML


In [2]:
map = folium.Map(location=[48, 8])
folium.Marker([48.2,8], popup="hier ist was").add_to(map)
map.save(outfile='einekarte.html')

In [3]:
%%HTML
<iframe width="100%" height ="350" src="einekarte.html"></iframe>

In [6]:
map_1 = folium.Map(location=[45.372, -121.6972], zoom_start=12,
                   tiles='Stamen Terrain')
folium.Marker([45.3288, -121.6625], popup='Mt. Hood Meadows').add_to(map_1)
folium.Marker([45.3311, -121.7113], popup='Timberline Lodge').add_to(map_1)
map_1.save('mthood.html')

In [7]:
%%HTML
<iframe width="100%" height ="350" src="http://83.212.239.107:8890/sparql"></iframe>

In [8]:
map_2 = folium.Map(location=[45.5236, -122.6750], tiles='Stamen Toner',
                   zoom_start=13)
folium.Marker(location=[45.5244, -122.6699], popup='The Waterfront').add_to(map_2)
folium.CircleMarker(location=[45.5215, -122.6261], radius=50,
                    popup='Laurelhurst Park', color='#3186cc',
                    fill_color='#3186cc').add_to(map_2)
map_2.save('portland.html')

In [9]:
%%HTML
<iframe width="100%" height ="350" src="portland.html"></iframe>

In [10]:
map_4 = folium.Map(location=[46.8527, -121.7649], tiles='Stamen Terrain',
                   zoom_start=13)
folium.Marker(location=[46.8354, -121.7325], popup='Camp Muir').add_to(map_4)
folium.ClickForMarker(popup='Waypoint').add_to(map_4)
map_4.save('mtrainier.html')

In [11]:
%%HTML
<iframe width="100%" height ="350" src="mtrainier.html"></iframe>

In [19]:
map_5 = folium.Map(location=[45.5236, -122.6750], zoom_start=13)
folium.RegularPolygonMarker(location=[45.5012, -122.6655], popup='Ross Island Bridge',
                   fill_color='#132b5e', number_of_sides=3, radius=10).add_to(map_5)
folium.RegularPolygonMarker(location=[45.5132, -122.6708], popup='Hawthorne Bridge',
                   fill_color='#45647d', number_of_sides=4, radius=10).add_to(map_5)
folium.RegularPolygonMarker(location=[45.5275, -122.6692], popup='Steel Bridge',
                   fill_color='#769d96', number_of_sides=6, radius=10).add_to(map_5)
folium.RegularPolygonMarker(location=[45.5318, -122.6745], popup='Broadway Bridge',
                   fill_color='#769d96', number_of_sides=8, radius=10).add_to(map_5)
map_5.save('bridges.html')

In [20]:
%%HTML
<iframe width="100%" height ="350" src="bridges.html"></iframe>

In [24]:
buoy_map = folium.Map(location=[46.3014, -123.7390], zoom_start=7,
                      tiles='Stamen Terrain')
popup1 = folium.Popup(max_width=800,
                     ).add_child(folium.Vega('vis2.json', width=500, height=250))
buoy_map.save(path='NOAA_buoys.html')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
import os
from folium.features import Template


class Map3d(folium.Map):

    def __init__(self, location=None, width='100%', height='100%', left='0%',
                 top='0%', position='relative', tiles='OpenStreetMap', API_key=None,
                 max_zoom=18, min_zoom=1, zoom_start=10, attr=None, min_lat=-90,
                 max_lat=90, min_lon=-180, max_lon=180, detect_retina=False, crs='EPSG3857'):
        super(Map3d, self).__init__(
            location=location, width=width, height=height,
            left=left, top=top, position=position, tiles=tiles,
            API_key=API_key, max_zoom=max_zoom, min_zoom=min_zoom,
            zoom_start=zoom_start, attr=attr, min_lat=min_lat,
            max_lat=max_lat, min_lon=min_lon, max_lon=max_lon,
            detect_retina=detect_retina, crs=crs
        )
        self._template = Template(u"""
        {% macro header(this, kwargs) %}
            <script src="https://www.webglearth.com/v2/api.js"></script>
            <style> #{{this.get_name()}} {
                position : {{this.position}};
                width : {{this.width[0]}}{{this.width[1]}};
                height: {{this.height[0]}}{{this.height[1]}};
                left: {{this.left[0]}}{{this.left[1]}};
                top: {{this.top[0]}}{{this.top[1]}};
                }
            </style>
        {% endmacro %}
        {% macro html(this, kwargs) %}
            <div class="folium-map" id="{{this.get_name()}}" ></div>
        {% endmacro %}

        {% macro script(this, kwargs) %}

            var southWest = L.latLng({{ this.min_lat }}, {{ this.min_lon }});
            var northEast = L.latLng({{ this.max_lat }}, {{ this.max_lon }});
            var bounds = L.latLngBounds(southWest, northEast);

            var {{this.get_name()}} = WE.map('{{this.get_name()}}', {
                                           center:[{{this.location[0]}},{{this.location[1]}}],
                                           zoom: {{this.zoom_start}},
                                           maxBounds: bounds,
                                           layers: [],
                                           crs: L.CRS.{{this.crs}}
                                         });
        {% endmacro %}
        """)

In [38]:
class TileLayer3d(folium.TileLayer):

    def __init__(self, tiles='OpenStreetMap', min_zoom=1, max_zoom=18, attr=None,
                 API_key=None, detect_retina=False, name=None, overlay=False, control=True):
        super(TileLayer3d, self).__init__(
            tiles=tiles, min_zoom=min_zoom, max_zoom=max_zoom,
            attr=attr, API_key=API_key, detect_retina=detect_retina,
            name=name, overlay=overlay, control=control
        )
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = WE.tileLayer(
                '{{this.tiles}}',
                {
                    maxZoom: {{this.max_zoom}},
                    minZoom: {{this.min_zoom}},
                    attribution: '{{this.attr}}',
                    detectRetina: {{this.detect_retina.__str__().lower()}}
                    }
                ).addTo({{this._parent.get_name()}});

        {% endmacro %}
        """)


In [39]:
m = Map3d(location=[-42, -42], tiles=None, zoom_start=2)
m.add_child(TileLayer3d(tiles='OpenStreetMap'))

url = ('http://services.arcgisonline.com/arcgis/rest/services'
       'Ocean/World_Ocean_Base'
       'MapServer/tile/{z}/{y}/{x}')

folium.TileLayer(
    tiles=url,
    name='World_Ocean_Base',
    attr='ESRI',
    overlay=True
).add_to(m)


m.save(os.path.join('results', 'Hacking_folium_3D_globe.html'))

m


In [40]:
import os
import folium

print(folium.__version__)

0.2.1


In [47]:
lat = +38.89399
lon = -77.03659
zoom_start = 17

m = folium.Map(location=[lat, lon], zoom_start=zoom_start)

kw = dict(fill_color='red', radius=20)
c0 = folium.CircleMarker([38.89415, -77.03738], **kw)
c1 = folium.CircleMarker([38.89415, -77.03578], **kw)


locations = [[(38.843596444352134, -77.03814983367920), (38.893379333722040, -77.03792452812195)],
             [(38.893379333722040, -77.03792452812195), (38.893162222428310, -77.03761339187622)],
             [(38.893162222428310, -77.03761339187622), (38.893028615148424, -77.03731298446655)],
             [(38.893028615148424, -77.03731298446655), (38.892920059048464, -77.03691601753235)],
             [(38.892920059048464, -77.03691601753235), (38.892903358095296, -77.03637957572937)],
             [(38.892903358095296, -77.03637957572937), (38.893011914220770, -77.03592896461487)],
             [(38.893011914220770, -77.03592896461487), (38.893162222428310, -77.03549981117249)],
             [(38.893162222428310, -77.03549981117249), (38.893404384982480, -77.03514575958252)],
             [(38.893404384982480, -77.03514575958252), (38.893596444352134, -77.03496336936950)]]

c3 = folium.PolyLine(locations=locations, color='orange', weight=20, opacity=1)

for c in [c0, c1, c3]:
    m.add_child(c)

m.save(os.path.join('results', 'MultiPolyline.html'))

m

## Linien ziehen mit Folium

Diese Funktion ist in Folium nicht gut dokumentiert und ich habe sie von Stack Overflow: [https://stackoverflow.com/questions/23905822/print-a-line-multiline-in-a-new-layer-over-a-map-using-folium]. Beachte dass das 2d Array, welches die Koordinaten hällt, nach [N, E] - Koordinaten sortiert ist, aber der von SPARQL übergebene WKT ist nach (E, N) sortiert. Das heißt, wir müssen das SPARQL Ergebnis erst umwandeln bevor wir es weiter nutzen können.

In [9]:
import folium

# Coordinates are 10 points on the great circle from Boston to
# San Francisco.
# Reference: http://williams.best.vwh.net/avform.htm#Intermediate
coordinates = [
    [42.3581, -71.0636],
    [42.82995815, -74.78991444],
    [43.17929819, -78.56603306],
    [43.40320216, -82.37774519],
    [43.49975489, -86.20965845],
    [41.4338549, -108.74485069],
    [40.67471747, -112.29609954],
    [39.8093434, -115.76190821],
    [38.84352776, -119.13665678],
    [37.7833, -122.4167]]

# Create the map and add the line
mappy = folium.Map(location=[41.9, -97.3], zoom_start=4)
my_PolyLine=folium.PolyLine(locations=coordinates,weight=1)
mappy.add_child(my_PolyLine)
mappy.save(outfile='einekartemitlinie.html')




In [10]:
%%HTML
<iframe width="100%" height ="350" src="einekartemitlinie.html"></iframe>

In [27]:

from geomet import wkt
point = {'type': 'Point', 'coordinates': [116.4, 45.2, 11.1]}
wkt.dumps(point, decimals=4)





'POINT (116.4000 45.2000 11.1000)'

In [32]:
import json

linestring = 'LINESTRING(2.379444 48.723333, 2.365278 48.720278, 2.2525 48.696111, 2.224167 48.69, 2.129167 48.652222, 2.093611 48.638056, 1.923056 48.669444, 1.805278 48.691111, 1.674167 48.715278, 1.516667 48.744167, 1.503611 48.746667, 1.341389 48.753889, 1.138611 48.763056, 0.895278 48.774167, 0.611389 48.786944, 0.530278 48.790556, 0.447222 48.743889, 0.291667 48.656111, 0.156667 48.58, -0.135278 48.415, -0.25 48.35, -0.380833 48.2725, -0.521667 48.189167, -1.5175 47.599722, -1.928889 47.348889, -2.363056 46.86, -3.867222 45.084167, -4.436111 44.4125, -6.521944 41.824444, -6.621944 41.700556, -8.123333 40.182778, -8.158889 40.146389, -8.287778 40.016389, -8.301944 40.001944, -8.311389 39.985556, -8.492778 39.665833, -8.679444 39.472778, -8.707222 39.444167, -8.817778 39.329722, -9.06 39.079444, -9.086111 39.038611, -9.133611 38.965, -9.186111 38.883333, -9.233333 38.809722, -9.285833 38.728056, -9.317222 38.678889, -9.331111 38.656389, -9.354444 38.619167, -9.391667 38.559167, -9.286111 38.520556, -9.248889 38.580278, -9.196389 38.669167, -9.1675 38.7175, -9.138889 38.766111, -9.134167 38.774167)'
# we will try to convert the wkt linestring into a latlon 2d array. Or GeoJSON.

somegeojson = wkt.loads(linestring)
with open('somegeo.json', 'w') as outfile:
    json.dump(somegeojson, outfile)
#somejsonstring = json.dumps(somegeojson)
#somejsonstring

In [31]:
map_3 = folium.Map(location=[40, 40], zoom_start=4)
folium.GeoJson(open('somegeo.json'), name='joergsgeojson').add_to(map_3)

#map_3.create_map(path='map_3.html')
folium.LayerControl().add_to(map_3)
map_3.save(outfile='einekartemitjson.html')

map_3

In [30]:
%%HTML
<iframe width="100%" height ="350" src="einekartemitjson.html"></iframe>

In [55]:
from geomet import wkt
import json

linestring = 'LINESTRING(2.379444 48.723333, 2.365278 48.720278, 2.2525 48.696111, 2.224167 48.69, 2.129167 48.652222, 2.093611 48.638056, 1.923056 48.669444, 1.805278 48.691111, 1.674167 48.715278, 1.516667 48.744167, 1.503611 48.746667, 1.341389 48.753889, 1.138611 48.763056, 0.895278 48.774167, 0.611389 48.786944, 0.530278 48.790556, 0.447222 48.743889, 0.291667 48.656111, 0.156667 48.58, -0.135278 48.415, -0.25 48.35, -0.380833 48.2725, -0.521667 48.189167, -1.5175 47.599722, -1.928889 47.348889, -2.363056 46.86, -3.867222 45.084167, -4.436111 44.4125, -6.521944 41.824444, -6.621944 41.700556, -8.123333 40.182778, -8.158889 40.146389, -8.287778 40.016389, -8.301944 40.001944, -8.311389 39.985556, -8.492778 39.665833, -8.679444 39.472778, -8.707222 39.444167, -8.817778 39.329722, -9.06 39.079444, -9.086111 39.038611, -9.133611 38.965, -9.186111 38.883333, -9.233333 38.809722, -9.285833 38.728056, -9.317222 38.678889, -9.331111 38.656389, -9.354444 38.619167, -9.391667 38.559167, -9.286111 38.520556, -9.248889 38.580278, -9.196389 38.669167, -9.1675 38.7175, -9.138889 38.766111, -9.134167 38.774167)'
linestring2 = 'LINESTRING(6.757222 51.280833, 6.768889 51.286389, 6.815278 51.309167, 6.850278 51.326111, 6.861944 51.331667, 6.883333 51.341111, 6.904444 51.350556, 6.928889 51.441944, 6.943611 51.496944, 6.813611 51.528056, 6.683889 51.559167, 6.670833 51.562222, 6.373889 51.466389, 6.348056 51.458056, 6.281667 51.345278, 6.271389 51.328056, 6.193056 51.193889, 6.143889 51.109722, 6.09 51.0175, 6.027222 50.995556, 5.989722 50.982222, 5.960556 50.971944, 5.894444 50.960278, 5.735833 50.932222, 5.5425 50.896944, 5.307778 50.854167, 5.045278 50.806389, 4.722222 50.745833, 4.601111 50.723056, 4.056667 50.618056, 3.832778 50.573889, 3.748333 50.449722, 3.675 50.342222, 3.318056 49.808333, 3.106389 49.485833, 2.191944 48.568611, 2.1 48.283333, 2.131667 48.133333, 2.179167 47.317778, 2.192222 47.089167, 2.265 46.964722, 2.3375 46.840278, 2.351944 46.815, 2.818056 45.998056, 2.837222 44.784444, 2.842222 44.454444, 2.852222 43.780556, 2.867222 42.750556, 2.832222 42.451389, 2.778056 41.985278, 2.771667 41.931389, 2.770278 41.913611, 2.751389 41.684444, 2.718889 40.899444, 2.714444 40.790556, 2.704444 40.545278, 2.7025 40.5, 2.7925 40.375556, 2.905278 40.219722, 2.78 40.055833, 2.630833 39.860556, 2.618889 39.845, 2.577778 39.790556, 2.460278 39.635, 2.395833 39.549444, 2.461389 39.513611, 2.508333 39.487778, 2.545833 39.467222, 2.576389 39.480556, 2.616944 39.498333, 2.738889 39.551667)'
# we will try to convert the wkt linestring into a latlon 2d array. Or GeoJSON.

somegeojson = wkt.loads(linestring)
somegeojson2 = wkt.loads(linestring2)
#with open('somegeo.json', 'w') as outfile:
#    json.dump(somegeojson, outfile)
#somejsonstring = json.dumps(somegeojson)
#somejsonstring
somegeojsonstring = json.dumps(somegeojson)
somegeojsonstring2 = json.dumps(somegeojson2)

somegeojsonstring

'{"type": "LineString", "coordinates": [[2.379444, 48.723333], [2.365278, 48.720278], [2.2525, 48.696111], [2.224167, 48.69], [2.129167, 48.652222], [2.093611, 48.638056], [1.923056, 48.669444], [1.805278, 48.691111], [1.674167, 48.715278], [1.516667, 48.744167], [1.503611, 48.746667], [1.341389, 48.753889], [1.138611, 48.763056], [0.895278, 48.774167], [0.611389, 48.786944], [0.530278, 48.790556], [0.447222, 48.743889], [0.291667, 48.656111], [0.156667, 48.58], [-0.135278, 48.415], [-0.25, 48.35], [-0.380833, 48.2725], [-0.521667, 48.189167], [-1.5175, 47.599722], [-1.928889, 47.348889], [-2.363056, 46.86], [-3.867222, 45.084167], [-4.436111, 44.4125], [-6.521944, 41.824444], [-6.621944, 41.700556], [-8.123333, 40.182778], [-8.158889, 40.146389], [-8.287778, 40.016389], [-8.301944, 40.001944], [-8.311389, 39.985556], [-8.492778, 39.665833], [-8.679444, 39.472778], [-8.707222, 39.444167], [-8.817778, 39.329722], [-9.06, 39.079444], [-9.086111, 39.038611], [-9.133611, 38.965], [-9.18611

In [82]:
map_4 = folium.Map(location=[40, 40], zoom_start=4, control_scale=True, prefer_canvas=True)

folium.GeoJson(somegeojsonstring, name='joergsgeojson', overlay=False,
              style_function=lambda feature: {'fillColor': '#ffaf00','color': 'black',
                                              'weight': 1.0,'dashArray': '1, 5'}
              ).add_to(map_4)

ab = folium.GeoJson(somegeojsonstring2, name='joergsgeojson', overlay=True).add_to(map_4)
ab.add_child(folium.Popup("some interesting Info about this trajectory"))
ab.add_to(map_4)

#map_3.create_map(path='map_3.html')
folium.LayerControl().add_to(map_4)
map_4.save(outfile='nochnekartemitjson.html')

map_4

In [58]:
linestring

'LINESTRING(2.379444 48.723333, 2.365278 48.720278, 2.2525 48.696111, 2.224167 48.69, 2.129167 48.652222, 2.093611 48.638056, 1.923056 48.669444, 1.805278 48.691111, 1.674167 48.715278, 1.516667 48.744167, 1.503611 48.746667, 1.341389 48.753889, 1.138611 48.763056, 0.895278 48.774167, 0.611389 48.786944, 0.530278 48.790556, 0.447222 48.743889, 0.291667 48.656111, 0.156667 48.58, -0.135278 48.415, -0.25 48.35, -0.380833 48.2725, -0.521667 48.189167, -1.5175 47.599722, -1.928889 47.348889, -2.363056 46.86, -3.867222 45.084167, -4.436111 44.4125, -6.521944 41.824444, -6.621944 41.700556, -8.123333 40.182778, -8.158889 40.146389, -8.287778 40.016389, -8.301944 40.001944, -8.311389 39.985556, -8.492778 39.665833, -8.679444 39.472778, -8.707222 39.444167, -8.817778 39.329722, -9.06 39.079444, -9.086111 39.038611, -9.133611 38.965, -9.186111 38.883333, -9.233333 38.809722, -9.285833 38.728056, -9.317222 38.678889, -9.331111 38.656389, -9.354444 38.619167, -9.391667 38.559167, -9.286111 38.520

In [75]:
l2 = 'LINESTRING(2.379444 48.723333, 2.365278 48.720278, 2.2525 48.696111, 2.224167 48.69, 2.129167 48.652222, 2.093611 48.638056)'
a = wkt.loads(l2)
a

{'coordinates': [[2.379444, 48.723333],
  [2.365278, 48.720278],
  [2.2525, 48.696111],
  [2.224167, 48.69],
  [2.129167, 48.652222],
  [2.093611, 48.638056]],
 'type': 'LineString'}

In [68]:
c1 = a['coordinates'][0][1]

c1




48.723333

In [74]:
arr = []
for point in a['coordinates']:
    arr.append(point)
    
arr

[[2.379444, 48.723333],
 [2.365278, 48.720278],
 [2.2525, 48.696111],
 [2.224167, 48.69],
 [2.129167, 48.652222],
 [2.093611, 48.638056],
 [1.923056, 48.669444],
 [1.805278, 48.691111],
 [1.674167, 48.715278]]

In [78]:
    #your WKT input:
    ls = 'LINESTRING(2.379444 48.723333, 2.365278 48.720278, 2.2525 48.696111, 2.224167 48.69, 2.129167 48.652222, 2.093611 48.638056)'

    #convert it to GeoJSON:
    ls_json = wkt.loads(ls)
    
    print(ls_json)

{'type': 'LineString', 'coordinates': [[2.379444, 48.723333], [2.365278, 48.720278], [2.2525, 48.696111], [2.224167, 48.69], [2.129167, 48.652222], [2.093611, 48.638056]]}


In [80]:
ls_json['coordinates'][5][2]

IndexError: list index out of range

In [81]:
import folium
from folium.plugins import TimestampedGeoJson
import numpy as np

coordinates = [[[[lon-8*np.sin(theta), -47+6*np.cos(theta)] for
                 theta in np.linspace(0, 2*np.pi, 25)],
                [[lon-4*np.sin(theta), -47+3*np.cos(theta)] for theta
                 in np.linspace(0, 2*np.pi, 25)]] for
               lon in np.linspace(-150, 150, 7)]
data = {
    "type": "FeatureCollection",
    "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [0, 0],
                    },
                "properties": {
                    "times": [1435708800000+12*86400000]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "MultiPoint",
                    "coordinates": [[lon, -25] for
                                    lon in np.linspace(-150, 150, 49)],
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 49)]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [[lon, 25] for
                                    lon in np.linspace(-150, 150, 25)],
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 25)]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "MultiLineString",
                    "coordinates": [[[lon-4*np.sin(theta),
                                      47+3*np.cos(theta)] for theta
                                     in np.linspace(0, 2*np.pi, 25)]
                                    for lon in
                                    np.linspace(-150, 150, 13)],
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 13)]
                    }
                },
            {
                "type": "Feature",
                "geometry": {
                    "type": "MultiPolygon",
                    "coordinates": coordinates,
                    },
                "properties": {
                    "times": [1435708800000+i*86400000 for
                              i in np.linspace(0, 25, 7)]
                    }
                },
        ],
    }

m = folium.Map([47, 3], zoom_start=1)
tgj = TimestampedGeoJson(data)
m.add_child(tgj)
m



In [87]:
import cesiumpy

v = cesiumpy.Viewer()
b = cesiumpy.Box(dimensions=(40e4, 30e4, 50e4), material=cesiumpy.color.RED, position=[-120, 40, 0])
v.entities.add(b)

v